In [1]:
import pandas as pd
from pathlib import Path

In [22]:
prophet_results = {}

for result_file in [
    "prophet_results_2014.csv",
    "prophet_results_default_2014.csv",
    "prophet_results_default_smp_2014.csv",
]:
    prophet_results[result_file] = pd.read_csv(Path("./") / result_file, index_col=0)

for result_file in prophet_results.keys():
    print(f"{result_file}: {prophet_results[result_file]['mape'].mean()}")

prophet_results_2014.csv: 26.3301940204345
prophet_results_default_2014.csv: 0.3435770094405776
prophet_results_default_smp_2014.csv: 0.3468237309926702


In [16]:
(prophet_results["prophet_results_2014.csv"]["mape"].min(),
prophet_results["prophet_results_2014.csv"]["mape"].max(),
prophet_results["prophet_results_2014.csv"]["mape"].mean())

(0.9605224438074478, 661.8497201295756, 26.3301940204345)

In [24]:
my_results = {}

for result_file in [
    "my_result_linear_2014.csv",
    # "my_result_same_2014.csv",
    # "my_result_simple_2014.csv",
]:
    my_results[result_file] = pd.read_csv(Path("./") / result_file, index_col=0)
    print(f"{result_file}: {my_results[result_file]['mape'].mean()}")

my_result_linear_2014.csv: 0.27082506370387294


In [66]:
my_results[result_file]

,mse,rmse,mae,mape
PRGO,0.060717,0.246408,0.211053,0.216011
INCY,0.168545,0.410543,0.367722,0.349816
ED,0.032185,0.179403,0.148304,0.140498
CDNS,0.005822,0.076303,0.055513,0.047230
SHW,0.065807,0.256529,0.223741,0.195271
...,...,...,...,...
FE,0.002496,0.049958,0.042216,0.045426
EVRG,0.012768,0.112996,0.101321,0.086837
UAL,0.335582,0.579295,0.512152,0.407554
DFS,0.008021,0.089560,0.066776,0.059575


In [25]:
index = my_results[result_file].index
prophet = []
my = []
for ticker in index:
    best_prophet = [results.loc[ticker]["mape"] for results in prophet_results.values()]
    best_my = [results.loc[ticker]["mape"] for results in my_results.values()]
    prophet.append(min(best_prophet))
    my.append(min(best_my))

In [26]:
sum(prophet) / len(prophet)

0.3311428929528097

In [27]:
sum(my) / len(my)

0.27082506370387294

In [39]:
len(prophet), len(my)

(437, 437)

In [46]:
min(best_prophet)

0.3126770053180203